In [1]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import xgboost as xgb
import pandas as pd
import numpy as np

In [2]:
df= pd.read_excel('Dimension_Assigned_Data.xlsx')
df = df.rename(columns={'Manual_Polarity':'Label'})

In [3]:
df['Label']=np.where(df['Label']==999,0,df['Label'])

In [4]:
X = df.drop(['Label'], axis=1)
y=df['Label']

In [5]:
corpus=[]
for i in X['Review']:
    corpus.append(i)

### TF-IDF

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer()
X = tf.fit_transform(corpus).toarray()

### Train Test Split

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

### Over-sampling

In [8]:
from imblearn.over_sampling import SMOTE
from collections import Counter
oversample = SMOTE()
X_train1, y_train1 = oversample.fit_resample(X_train, y_train)
from collections import Counter
print("The number of classes before fit {}".format(Counter(y_train)))
print("The number of classes after fit {}".format(Counter(y_train1)))

The number of classes before fit Counter({1: 262, 0: 115, -1: 81})
The number of classes after fit Counter({0: 262, 1: 262, -1: 262})


### Applying XGBoost classifier

In [9]:
model = XGBClassifier() 
model.fit(X_train1, y_train1)

C:\Users\Tanisha16\anaconda3\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:05:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [10]:
y_pred = model.predict(X_test) 
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions) 
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))
print("Accuracy: %.2f%%" % (accuracy * 100.0))

[[ 19   4  10]
 [  4  43  16]
 [  5  11 115]]
0.7797356828193832
              precision    recall  f1-score   support

          -1       0.68      0.58      0.62        33
           0       0.74      0.68      0.71        63
           1       0.82      0.88      0.85       131

    accuracy                           0.78       227
   macro avg       0.75      0.71      0.73       227
weighted avg       0.78      0.78      0.78       227

Accuracy: 77.97%
